In [1]:
!pip install pyarrow

In [2]:
import numpy as np
import os

base_path = r"C:\Users\ShinraS\Desktop\Projet_Challenge_BPI"

models_path = os.path.join(base_path, "models")

X_train = np.load(os.path.join(base_path, 'X_train.npy'))
y_act_train = np.load(os.path.join(base_path, 'y_act_train.npy'))
y_time_train = np.load(os.path.join(base_path, 'y_time_train.npy'))

X_test = np.load(os.path.join(models_path, 'X_test.npy'))

y_act_test = np.load(os.path.join(base_path, 'y_act_test.npy'))
y_time_test = np.load(os.path.join(base_path, 'y_time_test.npy'))

print(f"X_test shape: {X_test.shape}")

X_test shape: (209549, 5, 4)


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
import matplotlib.pyplot as plt

n_timesteps = X_train.shape[1] 
n_features = X_train.shape[2]  

num_classes = int(np.max(y_act_train) + 1) 


inputs_mono = Input(shape=(n_timesteps, n_features))
x = LSTM(64, return_sequences=False)(inputs_mono)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
output_act = Dense(num_classes, activation='softmax')(x)

model_mono_act = Model(inputs=inputs_mono, outputs=output_act)

model_mono_act.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


print(" Entraînement du modèle Mono-Activité")
history_act = model_mono_act.fit(
    X_train, y_act_train,
    validation_split=0.2,
    epochs=15, 
    batch_size=128,
    verbose=1
)

 Entraînement du modèle Mono-Activité
Epoch 1/15
5220/5220 [==============================] - 18s 3ms/step - loss: 0.7523 - accuracy: 0.7702 - val_loss: 0.4506 - val_accuracy: 0.8542
Epoch 2/15
5220/5220 [==============================] - 17s 3ms/step - loss: 0.4706 - accuracy: 0.8491 - val_loss: 0.4206 - val_accuracy: 0.8578
Epoch 3/15
5220/5220 [==============================] - 17s 3ms/step - loss: 0.4391 - accuracy: 0.8560 - val_loss: 0.3963 - val_accuracy: 0.8646
Epoch 4/15
5220/5220 [==============================] - 17s 3ms/step - loss: 0.4207 - accuracy: 0.8597 - val_loss: 0.3848 - val_accuracy: 0.8685
Epoch 5/15
5220/5220 [==============================] - 17s 3ms/step - loss: 0.4108 - accuracy: 0.8622 - val_loss: 0.3797 - val_accuracy: 0.8696
Epoch 6/15
5220/5220 [==============================] - 17s 3ms/step - loss: 0.4035 - accuracy: 0.8641 - val_loss: 0.3781 - val_accuracy: 0.8681
Epoch 7/15
5220/5220 [==============================] - 17s 3ms/step - loss: 0.3984 - accura

In [10]:

inputs_time = Input(shape=(n_timesteps, n_features))
x = LSTM(64, return_sequences=False)(inputs_time)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
output_time = Dense(1, activation='linear')(x) 
model_mono_time = Model(inputs=inputs_time, outputs=output_time)

model_mono_time.compile(
    optimizer='adam',
    loss='mse',    
    metrics=['mae']  
)


print("Entraînement du modèle Mono-Temps")
history_time = model_mono_time.fit(
    X_train, y_time_train,
    validation_split=0.2,
    epochs=15,
    batch_size=128,
    verbose=1
)

Entraînement du modèle Mono-Temps
Epoch 1/15
5220/5220 [==============================] - 19s 3ms/step - loss: 0.0176 - mae: 0.0771 - val_loss: 0.0119 - val_mae: 0.0530
Epoch 2/15
5220/5220 [==============================] - 18s 3ms/step - loss: 0.0116 - mae: 0.0547 - val_loss: 0.0101 - val_mae: 0.0475
Epoch 3/15
5220/5220 [==============================] - 17s 3ms/step - loss: 0.0106 - mae: 0.0501 - val_loss: 0.0097 - val_mae: 0.0475
Epoch 4/15
5220/5220 [==============================] - 17s 3ms/step - loss: 0.0103 - mae: 0.0485 - val_loss: 0.0097 - val_mae: 0.0453
Epoch 5/15
5220/5220 [==============================] - 16s 3ms/step - loss: 0.0101 - mae: 0.0473 - val_loss: 0.0095 - val_mae: 0.0447
Epoch 6/15
5220/5220 [==============================] - 16s 3ms/step - loss: 0.0100 - mae: 0.0467 - val_loss: 0.0095 - val_mae: 0.0445
Epoch 7/15
5220/5220 [==============================] - 16s 3ms/step - loss: 0.0098 - mae: 0.0461 - val_loss: 0.0094 - val_mae: 0.0443
Epoch 8/15
5220/5220 

In [11]:

inputs_multi = Input(shape=(n_timesteps, n_features))


shared_lstm = LSTM(128, return_sequences=False)(inputs_multi)
shared_dense = Dense(64, activation='relu')(shared_lstm)
shared_dropout = Dropout(0.2)(shared_dense)

# Sortie A : Activité (Classification)
output_act = Dense(num_classes, activation='softmax', name='act_output')(shared_dropout)

# Sortie B : Temps (Régression)
output_time = Dense(1, activation='linear', name='time_output')(shared_dropout)

model_multi = Model(inputs=inputs_multi, outputs=[output_act, output_time])

model_multi.compile(
    optimizer='adam',
    loss={'act_output': 'sparse_categorical_crossentropy', 'time_output': 'mse'},
    loss_weights={'act_output': 1.0, 'time_output': 1.0}, 
    metrics={'act_output': 'accuracy', 'time_output': 'mae'}
)

print("Entraînement du modèle Multi-Task")
history_multi = model_multi.fit(
    X_train, 
    {'act_output': y_act_train, 'time_output': y_time_train}, 
    validation_split=0.2,
    epochs=15,
    batch_size=128,
    verbose=1
)

Entraînement du modèle Multi-Task
Epoch 1/15
5220/5220 [==============================] - 40s 7ms/step - loss: 0.6752 - act_output_loss: 0.6523 - time_output_loss: 0.0229 - act_output_accuracy: 0.8013 - time_output_mae: 0.1044 - val_loss: 0.4231 - val_act_output_loss: 0.4082 - val_time_output_loss: 0.0149 - val_act_output_accuracy: 0.8622 - val_time_output_mae: 0.0731
Epoch 2/15
5220/5220 [==============================] - 37s 7ms/step - loss: 0.4432 - act_output_loss: 0.4264 - time_output_loss: 0.0168 - act_output_accuracy: 0.8601 - time_output_mae: 0.0848 - val_loss: 0.3952 - val_act_output_loss: 0.3820 - val_time_output_loss: 0.0132 - val_act_output_accuracy: 0.8688 - val_time_output_mae: 0.0677
Epoch 3/15
5220/5220 [==============================] - 38s 7ms/step - loss: 0.4189 - act_output_loss: 0.4031 - time_output_loss: 0.0158 - act_output_accuracy: 0.8647 - time_output_mae: 0.0798 - val_loss: 0.3861 - val_act_output_loss: 0.3732 - val_time_output_loss: 0.0129 - val_act_output_ac

In [16]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np

all_indices = np.arange(26)

le_act = LabelEncoder()
placeholder_names = [f"Activity_{i}" for i in range(26)]
le_act.fit(placeholder_names)

scaler_time = MinMaxScaler()
scaler_time.fit(np.array([0, 15]).reshape(-1, 1)) 

print("'le_act' est prêt avec 26 classes")

'le_act' est prêt avec 26 classes


In [6]:
import joblib
import tensorflow as tf
import os
path_scaler = os.path.join("..", "models", "scaler_time.joblib")
scaler_time = joblib.load(path_scaler)
path_model = os.path.join("..", "models", "model_multi_task.keras")
model = tf.keras.models.load_model(path_model)
preds_act, preds_time_norm = model.predict(X_test)
pred_act_idx = np.argmax(preds_act, axis=1)
pred_act_names = [f"Act_{i}" for i in pred_act_idx] 

preds_time_log = scaler_time.inverse_transform(preds_time_norm.reshape(-1, 1))
preds_time_hours = np.expm1(preds_time_log).flatten()

import pandas as pd
analysis_df = pd.DataFrame({
    'Activite_Predite_Index': pred_act_idx,
    'Delais_Heures_Predits': preds_time_hours
})

SEUIL_CRITIQUE = 21.0
analysis_df['Est_Goulot'] = analysis_df['Delais_Heures_Predits'] > SEUIL_CRITIQUE
taux_alerte = analysis_df['Est_Goulot'].mean() * 100
print(f"Taux d'alerte : {taux_alerte:.2f}%")
print(analysis_df.head())

c:\Users\ShinraS\anaconda3\envs\tf_wbc\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.7.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


6549/6549 [==============================] - 9s 1ms/step
Taux d'alerte : 9.28%
   Activite_Predite_Index  Delais_Heures_Predits  Est_Goulot
0                       3              -0.044982       False
1                      21              24.637098        True
2                      12              -0.081590       False
3                      13              -0.147051       False
4                      16               0.104082       False


In [16]:
import os
import joblib
le_act = joblib.load('../models/le_act.joblib')
if not os.path.exists('../models'):
    os.makedirs('../models')
model.save('../models/model_multi_task.keras')
joblib.dump(le_act, '../models/le_act.joblib')
joblib.dump(scaler_time, '../models/scaler_time.joblib')
np.save('../models/X_test.npy', X_test)

print("All saved")

All saved
